In [1]:
# using the Resnet50 model as a feature extraction model
import pickle
import cv2
import pandas as pd

from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input
from keras.applications.resnet import decode_predictions
from keras.models import Model


import logging
logger = logging.getLogger()  # Logger

logger.setLevel("INFO")
#logging.info("Hello")

In [2]:
def write_pickle(path, obj):
    with open(path, "wb") as f:
        pickle.dump(obj, f)
        
def read_pickle(path):
    with open(path, "rb") as f:
        obj = pickle.load(f)
    return obj

In [3]:
filtered_img_dict = read_pickle(path="/mnt/DataRAID/melismail/Preprocessing/data/pickle_data/Preprocessing/filtered_img_dict.pkl")

In [6]:
def image_feature(img, model):
    img = cv2.resize(img, (224, 224))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape(1, 224, 224, 3)
    img = preprocess_input(img)
    features = model.predict(img)
    return features

In [7]:
logging.info("LETS GOOOO minus eins")
model = ResNet50()
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)

features_img_dict = {k: image_feature(img=v, model=model)[0]
                    for k, v in filtered_img_dict.items()}
logging.info("DOOOONE")

INFO:root:LETS GOOOO minus eins
2022-08-28 12:46:00.492877: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8100
2022-08-28 12:46:00.869593: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-28 12:46:00.870528: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-28 12:46:00.870547: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2022-08-28 12:46:00.871327: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-08-28 12:46:00.871373: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
INFO:root:DOOOONE


In [8]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_2[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                            

In [9]:
feature_df = pd.DataFrame(features_img_dict.items(), columns=['Img', 'Features'])

In [10]:
print(feature_df)

                   Img                                           Features
0       (18QH2FGR4, 0)  [0.7959027, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1641702...
1       (18QH2FGR4, 1)  [0.22671106, 0.0, 0.0, 0.0, 0.0, 0.0, 0.315824...
2       (18QH2FGR4, 2)  [0.7223637, 0.0, 0.009048088, 0.0, 0.059443235...
3       (18QH2FGR4, 3)  [0.24703209, 0.0038301263, 0.0, 0.0, 0.0397417...
4       (18QH2FGR4, 4)  [0.31141222, 0.016817218, 0.0, 0.0, 0.03273587...
...                ...                                                ...
6056  (ILH0VMVF5, 831)  [0.0, 0.030632433, 0.0, 0.0, 0.0, 0.0, 0.14051...
6057  (ILH0VMVF5, 832)  [0.0039850953, 0.040655117, 0.0, 0.0, 0.0, 0.0...
6058  (ILH0VMVF5, 833)  [0.036934033, 0.06299035, 0.0, 0.06800861, 0.0...
6059  (ILH0VMVF5, 834)  [0.016906105, 0.044178925, 0.0, 0.0102700265, ...
6060  (ILH0VMVF5, 835)  [0.3400179, 0.12037215, 0.011650665, 0.0, 0.0,...

[6061 rows x 2 columns]


In [15]:
write_pickle(path="/mnt/DataRAID/melismail/Preprocessing/data/pickle_data/Preprocessing/features_res_without_top_img_dict.pkl", obj=features_img_dict)
write_pickle(path="/mnt/DataRAID/melismail/Preprocessing/data/pickle_data/Preprocessing/features_res_without_top_img_df.pkl", obj=feature_df)